In [2]:
import mysql.connector
import pandas as pd 
import numpy as np 

C:\Users\chanks\AppData\Local\Temp\ipykernel_1748\1685290688.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


NotSupportedError: Authentication plugin 'caching_sha2_password' is not supported

In [5]:
import mysql.connector

# Replace these values with your actual database credentials
host = 'localhost'
database = 'grocery'
user = 'chanks'
password = 'milkyway'

try:
    # Establish a connection to the MySQL server
    connection = mysql.connector.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    if connection.is_connected():
        print("Connected to MySQL database")

        # Your database operations go here

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the connection in the finally block to ensure it's always closed
    if 'connection' in locals() and connection.is_connected():
        connection.close()
        print("Connection closed")

Error: Authentication plugin 'caching_sha2_password' is not supported
